## Conway's Game of Life with Numpy

In his solution to the Game Of Life, Jake VanderPlas offers two algorithms for generating the successive states of the array, the first of which uses Numpy to succinctly count neighboring cells and follow the games rules.  You can see his Jupyter notebook for this at:
http://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/

For an excellent history and discussion, see the Wikipedia article.

Jake call's his solution "simple" though the step algorithm is not immediately clear.  In one of the comments to his page, a reader calls his code "ugly" perhaps because it is not intuitive and therefore not "pythonic".  The purpose here is to explain how his solution works, comment on efficiency and then attempt to qualify the solution as to pythonicity and attractiveness. 

The part of his code in question is:


In [1]:
import numpy as np

def life_step_1(X):
    """Game of life step using generator expressions"""
    nbrs_count = sum(np.roll(np.roll(X, i, 0), j, 1)
                     for i in (-1, 0, 1) for j in (-1, 0, 1)
                     if (i != 0 or j != 0))
    return (nbrs_count == 3) | (X & (nbrs_count == 2))

This is the "toroidal" version of the Game where the top and bottom wrap around as do the left and right edges, so the grid size remains fixed but the edges allow for all neighbors and infinite wrapping.

In the usual solution to the Game, each cell in an n x m grid is separately evaluated to determine which of the adjacent 8 cells are active ("live") or empty ("dead"). 

The life_step_1 function takes a numpy array as a parameter with the cells being either True or False.  Operations are done directly on the array not on individual cells.  The first line of the function's code is the generator expression using the embedded np.roll method calls.  np.roll() rotates an array contents a given number of rows or columns, forwards or backwards. In this case, the combined effect is to roll the grid 8 times to each possible adjacency, each time creating a new array of the same size.  The sum of the 8 arrays counts -- imagine them stacked and counting in the third dimension -- is an array of the same side where each cell now countains the count of adjacent True cells.  The trick is to reconceptualize the np.roll as the notation of the existence of adjacencies;  the point of view also switches from the row/column being moved to the fixed cell locations as registering counts.  Jake doesn't bother to recognize this bit of cleverness.  And to make this even more painfully clear, let's take a simple example of a 3 x 3 grid with a single True cell in the middle:

0 0 0 <br>
0 1 0 <br>
0 0 0 <br>

The eight rolled arrays are:

1 0 0 ... 0 1 0 ... 0 0 1 ... 0 0 0 ... 0 0 0 ... 0 0 0 ... 0 0 0 ... 0 0 0 <br>
0 0 0 ... 0 0 0 ... 0 0 0 ... 1 0 0 ... 0 0 1 ... 0 0 0 ... 0 0 0 ... 0 0 0 <br>
0 0 0 ... 0 0 0 ... 0 0 0 ... 0 0 0 ... 0 0 0 ... 1 0 0 ... 0 1 0 ... 0 1 0 <br>

When summed, this gives the following array of neighbor counts (not booleans):

1 1 1 <br>
1 0 1 <br>
1 1 1 <br>


The second expression in the function is the return statement that applies the Games rules to the neighbor count array via numpy broadcasting.  The left side (nbrs_count == 3) evaluates each cell in the array to generate a new boolean array of the same size.  This is the Game rule for generating a new live cell when the neighbor count is 3.  In the 3 x 3 example, the result is clearly all False cells since the counts are only 0 or 1.  The right side tests a similar boolean array where the count is 2, which corresponds to the Game rule to keep a cell alive hence this array is &'ed against the original matrix.  The True values from both results are combined via "|" to create a new array which is returned as the next step in the Game which is evaluated in term by the same function, and so on.  In the 3 x 3 example, there are no live cells and the Game ends.

The following code unpacks Jake's life_step_1 function and prints out all the intervening states.

In [2]:
def printM(X):
    '''
    Display a boolean matrix
    '''
    print('__________________')
    for i in range(X.shape[0]):
        print('        ', end='')
        for j in range(X.shape[1]):
            if X[i,j] == True:
                print('■', end='') 
            else:
                print('∙', end='') 
        print('')          
    print('__________________')
    

def life_step(X):
    
    rjlist = []
    
    for i in (-1, 0, 1):
        print('ri {}'.format(i))
        ri = np.roll(X, i, 0)
        printM(ri)
        for j in (-1, 0, 1):
            if (i != 0 or j != 0):                               
                rj = np.roll(ri, j, 1)
                print('rj {} {}'.format(i, j))
                printM(rj)
                rjlist.append(rj) 
                print('')    
    nbrs_M = sum(rjlist)
    
    print('neighbor counts:')
    print(nbrs_M)
    print('')
    
    to_generate = (nbrs_M == 3)
    print('to generate:')
    printM(to_generate)
    
    to_stay = (X & (nbrs_M == 2))
    print('to_stay:')
    printM(to_stay)
    
    res = to_generate | to_stay
    print('res:')
    printM(res)
    print('')
                     
    return res 



The following code gives an example with four iterations ending in a state with 4 live cells that will repeat ad infinitum.

In [3]:
X = np.zeros((4, 4), dtype=bool)
X[1, 1] = True
X[1, 2] = True
X[2, 2] = True

print('start:')
printM(X)
print('')

for i in range(3):
    X = life_step(X)

print('end:')
printM(X)
print('')


start:
__________________
        ∙∙∙∙
        ∙■■∙
        ∙∙■∙
        ∙∙∙∙
__________________

ri -1
__________________
        ∙■■∙
        ∙∙■∙
        ∙∙∙∙
        ∙∙∙∙
__________________
rj -1 -1
__________________
        ■■∙∙
        ∙■∙∙
        ∙∙∙∙
        ∙∙∙∙
__________________

rj -1 0
__________________
        ∙■■∙
        ∙∙■∙
        ∙∙∙∙
        ∙∙∙∙
__________________

rj -1 1
__________________
        ∙∙■■
        ∙∙∙■
        ∙∙∙∙
        ∙∙∙∙
__________________

ri 0
__________________
        ∙∙∙∙
        ∙■■∙
        ∙∙■∙
        ∙∙∙∙
__________________
rj 0 -1
__________________
        ∙∙∙∙
        ■■∙∙
        ∙■∙∙
        ∙∙∙∙
__________________

rj 0 1
__________________
        ∙∙∙∙
        ∙∙■■
        ∙∙∙■
        ∙∙∙∙
__________________

ri 1
__________________
        ∙∙∙∙
        ∙∙∙∙
        ∙■■∙
        ∙∙■∙
__________________
rj 1 -1
__________________
        ∙∙∙∙
        ∙∙∙∙
        ■■∙∙
        ∙■∙∙
__________________

rj 1 0
_______________

That the neighbor calculations are done so succinctly and against arrays as opposed to a cell-by-cell basis is compelling.  Jake says that this method is not "extremely performant" and "will not work well for large problems with many time steps".   